# Using GPano.py

In [ ]:
import sys  
sys.path.insert(0, r'/media/huan/HD4T/Research/StreetView/Google_street_view')
import GPano
# import Fill_sidewalks as sw
import pandas as pd
import os
import glob

gpano = GPano.GPano()
gsv = GPano.GSV_depthmap()


def list_to_file(a_list, file_name):
    with open(file_name, mode='w', encoding='utf-8') as myfile:
        myfile.write('\n'.join(a_list))
        myfile.write('\n')

In [ ]:
panoId_map = r'/media/huan/SSD/Datasets/ocean_city/yolo/inference/rename.csv'
depth_map_dir = r'/media/huan/SSD/Datasets/ocean_city/yolo/inference/depthmap'

detected_dir = r'/media/huan/SSD/Datasets/ocean_city/yolo/inference/output'



if not os.path.exists(depth_map_dir):
    os.mkdir(depth_map_dir)

In [ ]:
panoId_file = r'/media/huan/SSD/Datasets/ocean_city/yolo/inference/rename.csv'

In [ ]:
panoId_df = pd.read_csv(panoId_file)

In [ ]:
panoId_dict = {}
for idx, row in panoId_df.iterrows():
    jpg = row['jpg']
    panoId = row['panoid']
    jpg_pos = panoId.find(".jpg")
    panoId_dict[jpg] = panoId[:jpg_pos]
 

In [ ]:
jpgs = glob.glob(os.path.join(detected_dir, "*.jpg"))
distances = []
for jpg in jpgs:
    basename = os.path.basename(jpg)
    panoId = panoId_dict[basename]
#     jdata = gpano.getJsonfrmPanoID(panoId, dm=1, saved_path=depth_map_dir)
#     dm = gsv.getDepthmapfrmJson(jdata)
    
    img = gsv.saveDepthMap_frm_panoId(panoId, saved_path=depth_map_dir)
    shape = img.size
#     print(img)
    lowest_row = max(0, shape[1] - 1)
    central_col = int(shape[0]/2)
    print(lowest_row, central_col)
    d = img.load()[central_col, lowest_row]
    line = ",".join([panoId, str(d)])
    distances.append(line)

In [ ]:
list_to_file(jpgs, os.path.join(r"/media/huan/SSD/Datasets/ocean_city/yolo/inference/jpgs.txt"))
list_to_file([str(d) for d in distances], os.path.join(r"/media/huan/SSD/Datasets/ocean_city/yolo/inference/distances.txt"))

# Calculate the height


## rowcol_to_spherial


In [36]:
from math import *

def castesian_to_shperical(col, row, fov_h, height, width):  # yaw: set the heading, pitch
    """
    Convert the row, col to the  spherical coordinates 
    :param row, cols: 
    :param fov_h:
    :param height:
    :param width:
    :return:
    """
     
    col = col - width/2  # move the origin to center
    row = height/2 -row
 
    fov_v = atan((height * tan((fov_h / 2)) / width)) * 2
     
    
    r = (width/2)/tan(fov_h/2)
     
    s = sqrt(col**2 + r**2)
    
    theta = atan(row/s)
    phi = atan(col/r)
    
#     print("fov_v, r, s:", fov_v, r, s)
     
    return phi, theta

 

In [30]:
import glob
import os
from PIL import Image

dets_dir = r'/media/huan/SSD/Datasets/ocean_city/yolo/inference/output'
dets_txts = glob.glob(os.path.join(dets_dir, "*.txt"))


In [38]:
target_id = 0

for txt in dets_txts[:2]:
    print(txt)
    f = open(txt, 'r')
    lines = f.readlines()
#     print(lines)
    lines = [line.replace(" \n", "") for line in lines]
#     print(lines)
    objects_fileds = [line.split(' ') for line in lines]
#     print(objects_fileds)

    fov_h = math.pi/2

    img_name = txt.replace(".txt", '.jpg')
    img = Image.open(img_name)
    
    img_w, img_h = img.size
#     print(img_w, img_h)
    
    for o in objects_fileds:
        obj_id = int(o[0])
        xywh = o[1:]
        x, y, w, h = [float(n) for n in xywh]
#     print(obj_id, x, y, w, h)
        if obj_id == target_id:
    
            top_row = img_h * (y - h/2)
            bottom_row = img_h * (y + h/2)
            col = img_w * col
            
            top_agl = castesian_to_shperical(col, top_row, fov_h, img_h, img_w)
            bottom_agl = castesian_to_shperical(col, bottom_row, fov_h, img_h, img_w)
            
            print("top_row, bottom_row, top_agl, bottom_agl:", top_row, bottom_row, top_agl, bottom_agl)
            print('\n')
        
        
    
    
    

/media/huan/SSD/Datasets/ocean_city/yolo/inference/output/13785.txt
/media/huan/SSD/Datasets/ocean_city/yolo/inference/output/13786.txt
top_row, bottom_row, top_agl, bottom_agl: 1581.9994112 1779.9994368 (1.5707963267948966, 1.2163431229498649e-26) (1.5707963267948966, -1.720659806015166e-26)


top_row, bottom_row, top_agl, bottom_agl: 1647.9993087999999 1842.9994751999998 (1.5707963267948966, 7.13172452345939e-31) (1.5707963267948966, -7.978248758217332e-30)


top_row, bottom_row, top_agl, bottom_agl: 1654.0011904000003 1843.0013056000003 (1.5707963267948966, 1.3391240088744936e-34) (1.5707963267948966, -2.3973348382276136e-33)




In [5]:
import math

col = 1491
  
row = 2101
# row = 1856
fov_h = math.pi/2
height = 4096
width = 4096
castesian_to_shperical(col, row, fov_h, height, width)

fov_v, r, s: 1.5707963267948966 2048.0000000000005 2122.393224640524


(-0.26554954089837773, -0.024966621041549453)